## Preambule

In [42]:
import numpy as np
import xarray as xr
from importlib import reload
import Plotfunctions
import pandas as pd
from plotly.subplots import make_subplots
import os
import matplotlib as mpl
import plotly.io
import matplotlib.pyplot as plt
from datetime import date
reload(Plotfunctions)
import plotly.graph_objects as go
from pathlib import Path
from Plotfunctions import rescale_size, triangleplot, tableplot, spaghettiplot, draw_line
path = Path.cwd() / "Data"

## Read data files

In [4]:
try:
    XRvar.close()
except:
    3
XRvar = xr.open_dataset(path / "Variances.nc")

In [5]:
Varlist = pd.read_excel(path / "Varlist_noadd.xlsx", sheet_name = "Data")
vars = np.array(Varlist["Variable"])
cats = np.array(Varlist["Category"])
Names = np.unique(cats)

## Final figure

In [6]:
XRraw = xr.open_dataset(path / "XRdata.nc")
XRmeta = xr.open_dataset(path / "XRmeta.nc")
DF = pd.read_csv(path / "Models.csv")
values = np.array(XRraw.Value)
modscens = np.array(XRraw.ModelScenario)
mods = np.array([i.split('|')[0] for i in modscens])
ccat = np.array(XRmeta.sel(ModelScenario=XRraw.ModelScenario).Category.data)
unimods = np.unique(mods)
uniccat = np.unique(ccat)

Define table

In [7]:
XR = XRvar
t = 2050
srt = 1
from plotly.colors import n_colors
import plotly.express as px

In [8]:
tot = np.array(XR.sel(Time=2050).S_c)+np.array(XR.sel(Time=2050).S_z)+np.array(XR.sel(Time=2050).S_m)
ar = np.array([np.array(XR.Variable),
np.array(XR.sel(Time=t).S_m/tot).round(2),
np.array(XR.sel(Time=t).S_c/tot).round(2),
np.array(XR.sel(Time=t).S_z/tot).round(2)])

colors = n_colors('rgb(255, 200, 200)', 'rgb(200, 0, 0)', 100, colortype='rgb')
bar = px.colors.diverging.RdBu_r
colors = []
for i in range(len(bar)-1):
        colors = colors+n_colors(px.colors.diverging.RdBu_r[i], px.colors.diverging.RdBu_r[i+1], int(100/len(bar)), colortype='rgb')
colors = colors+n_colors(px.colors.diverging.RdBu_r[i+1], px.colors.diverging.RdBu_r[i+1], int(100/len(bar)), colortype='rgb')
ar100 = (ar[1:]*100).astype(int)

In [9]:
table1 = go.Table(
columnwidth = [70,40, 40, 40],
header = dict(
        values = [['<b>Variable</b><br>in '+str(t)],
                ['Model'],
                ['Climate category'],
                ['Scenario']],

        line_color='darkslategray',
        fill_color='black',
        align=['center','center','center','center'],
        font=dict(color='white', size=12),
        height=40
),
cells=dict(
        values=ar[:, ar[srt].argsort()[::-1]],
        line_color='black',
        align=['left','center','center','center'],
        fill_color=['white',
                np.array(colors)[ar100[:, ar[srt].argsort()[::-1]][0]],
                np.array(colors)[ar100[:, ar[srt].argsort()[::-1]][1]],
                np.array(colors)[ar100[:, ar[srt].argsort()[::-1]][2]]],
        font_size=12,
        font={'family' :["Arial", "Arial Black", "Arial Black", "Arial Black"],
        'color': 'black'},
        height=30)
        )

In [10]:
XR = XRvar.sel(Variable = vars[cats == "Primary Energy"])
DF_counts = pd.read_csv("X:/user/dekkerm/Projects/AR6_Variance/variancedecomposition/Data/Counts.csv", index_col=0)
XR = XR.sel(Time=range(2030,2101))
varlist = XR.Variable.data
years = XR.Time.data
varmax = np.max(XR['Var_total']).data
cols = ['forestgreen', 'tomato', 'steelblue', 'goldenrod', 'purple', 'grey', 'brown',
        'magenta', 'red', 'darkgrey', 'blue', 'black', 'darkgreen']
years_str = np.copy(years).astype(str)
years_str[(years_str != '2050') & (years_str != '2100')] = ''

## Optional figure - Spaghetti / intro

In [9]:
Figure_spagh = spaghettiplot(XRraw, XRmeta, "Final Energy|Industry", "Emissions|CO2")
plotly.io.write_image(Figure_spagh, Path.cwd() / 'Figures/Paperfigures/Figure_1.pdf', format='pdf')

## Figure 2 - Triangle Electricity

In [43]:
xrsub = XRvar.sel(Variable = vars[cats == 'Electricity Generation'][:-1])
Figure_2 = triangleplot(xrsub, XRraw)

In [45]:
Figure_2

## Figure 3 - Triangle Industry

In [44]:
xrsub = XRvar.sel(Variable = vars[cats == 'Sector Industry'][:-1])
Figure_3 = triangleplot(xrsub, XRraw)

## Full Table / overview

In [31]:
srt = 1 # Sorting: 1 = model, 2 = climate
Figure_4 = tableplot(XRvar, 2050, srt)


In [14]:
XR = XRvar
t = 2050
srt = 1
from plotly.colors import n_colors
import plotly.express as px

In [15]:
tot = np.array(XR.sel(Time=2050).mean(dim='Member').S_c)+np.array(XR.sel(Time=2050).mean(dim='Member').S_z)+np.array(XR.sel(Time=2050).mean(dim='Member').S_m)
ar = np.array([np.array(XR.Variable),
np.array(XR.sel(Time=t).mean(dim='Member').S_m/tot).round(2),
np.array(XR.sel(Time=t).mean(dim='Member').S_c/tot).round(2),
np.array(XR.sel(Time=t).mean(dim='Member').S_z/tot).round(2)])

colors = n_colors('rgb(255, 200, 200)', 'rgb(200, 0, 0)', 100, colortype='rgb')
bar = px.colors.diverging.RdBu_r
colors = []
for i in range(len(bar)-1):
        colors = colors+n_colors(px.colors.diverging.RdBu_r[i], px.colors.diverging.RdBu_r[i+1], int(100/len(bar)), colortype='rgb')
colors = colors+n_colors(px.colors.diverging.RdBu_r[i+1], px.colors.diverging.RdBu_r[i+1], int(100/len(bar)), colortype='rgb')
ar100 = (ar[1:]*100).astype(int)
fig = go.Figure(data=[go.Table(
columnwidth = [70,40, 40, 40],
header = dict(
        values = [['<b>Variable</b><br>in '+str(t)],
                ['Model'],
                ['Climate category'],
                ['Scenario']],

        line_color='darkslategray',
        fill_color='black',
        align=['center','center','center','center'],
        font=dict(color='white', size=12),
        height=40
),
cells=dict(
        values=ar[:, ar[srt].argsort()[::-1]],
        line_color='black',
        align=['left','center','center','center'],
        fill_color=['white',
                np.array(colors)[ar100[:, ar[srt].argsort()[::-1]][0]],
                np.array(colors)[ar100[:, ar[srt].argsort()[::-1]][1]],
                np.array(colors)[ar100[:, ar[srt].argsort()[::-1]][2]]],
        font_size=12,
        font={'family' :["Arial", "Arial Black", "Arial Black", "Arial Black"],
        'color': 'black'},
        height=30)
        )
])
fig.update_layout(height=2000, width=1400)

## Savings

In [40]:
try:
    os.remove('Figures/Paperfigures/Figure_1.html')
except:
    3
def html_w(typ):
    return '<html> '+typ+' <p style="font-family: Arial">'

today = date.today()
td = today.strftime("%B %d, %Y")

with open('Figures/Paperfigures/Figure_1.html', 'a') as f:
    f.write(html_w('<h1>')+'Figure 1 - Electricity Generation</p></h1>')
    f.write(html_w('<body>')+'Version: '+td+'</p></body>')
    f.write(Figure_2.to_html(full_html=False, include_plotlyjs='cdn'))

In [41]:
try:
    os.remove('Figures/Paperfigures/Figure_2.html')
except:
    3
def html_w(typ):
    return '<html> '+typ+' <p style="font-family: Arial">'

today = date.today()
td = today.strftime("%B %d, %Y")

with open('Figures/Paperfigures/Figure_2.html', 'a') as f:
    f.write(html_w('<h1>')+'Figure 2 - Sector Industry</p></h1>')
    f.write(html_w('<body>')+'Version: '+td+'</p></body>')
    f.write(Figure_3.to_html(full_html=False, include_plotlyjs='cdn'))

In [33]:
try:
    os.remove('Figures/Paperfigures/Paperplots.html')
except:
    3
def html_w(typ):
    return '<html> '+typ+' <p style="font-family: Arial">'

today = date.today()
td = today.strftime("%B %d, %Y")
figs = [Figure_2, Figure_3]
fignames = ['Triangle Electricity', 'Triangle Industry']

with open('Figures/Paperfigures/Paperplots.html', 'a') as f:
    f.write(html_w('<h1>')+'Figures version: '+td+'</p></h1>')
    
    for n_i in range(len(figs)):
        if n_i > 0:
            f.write('<hr>')
        f.write(html_w('<h1>')+'Figure '+str(n_i+1)+' - '+fignames[n_i]+'</p></h1>')
        f.write(figs[n_i].to_html(full_html=False, include_plotlyjs='cdn'))